In [9]:
import random
import numpy as np
import PDSVRP_instance
import copy

In [10]:
import import_ipynb
from importnb import Notebook

with Notebook():
    import heuristic as h

without ls

In [11]:
def initial_solution_construction_no_LS(instance, w1, w2, w3, w4, w5, gamma):
    A = [c for c in range (1, instance.N)]
    solution = h.recreate(instance, [[[[] for _ in range(instance.h)], [[] for _ in range(instance.D)]],A], w1, w2, w3, w4, w5, gamma)
    return solution

In [12]:
def SISSRs_no_LS(instance, sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma, n_nearest, delta, epsilon, iter_imp, iter_max, p_min, p_max, max_unfeasible_swaps_perturb):
    s_0 = initial_solution_construction_no_LS(instance, w1, w2, w3, w4, w5, gamma)
    s_curr = s_0
    s_best = s_0
    iterations_without_improvement = 0
    iteration_counter = 0
    while (iteration_counter < iter_max):
        s = h.ruin_and_recreate(instance, copy.deepcopy(s_curr), sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma)
        if h.cost(instance, s) < h.cost(instance, s_curr)*(1+delta):
            s_curr = s
            if h.cost(instance, s_curr) < h.cost(instance, s_best):
                s_best = s_curr
                iterations_without_improvement = 0
            else:
                iterations_without_improvement += 1
        if iterations_without_improvement >= iter_imp:
            s_curr = h.perturbate(instance, s_curr, p_min, p_max, max_unfeasible_swaps_perturb)
            iterations_without_improvement = 0
        delta = delta * epsilon
        iteration_counter+=1
        print("Iteration: ", iteration_counter)
        print("Costo miglior soluzione: ", h.cost(instance, s_best))

    return s_best

no Threshold acceptence

In [13]:
def SISSRs_no_TA(instance, sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma, n_nearest, delta, epsilon, iter_imp, iter_max, p_min, p_max, max_unfeasible_swaps_perturb):
    s_0 = h.initial_solution_construction(instance, w1, w2, w3, w4, w5, gamma, n_nearest)
    s_curr = s_0
    s_best = s_0
    iterations_without_improvement = 0
    iteration_counter = 0
    while (iteration_counter < iter_max):
        s = h.ruin_and_recreate(instance, copy.deepcopy(s_curr), sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma)
        if h.cost(instance, s) < h.cost(instance, s_curr):
            s_curr = h.local_search(instance, s, n_nearest)
            if h.cost(instance, s_curr) < h.cost(instance, s_best):
                s_best = s_curr
                iterations_without_improvement = 0
            else:
                iterations_without_improvement += 1
        if iterations_without_improvement >= iter_imp:
            s_curr = h.perturbate(instance, s_curr, p_min, p_max, max_unfeasible_swaps_perturb)
            iterations_without_improvement = 0
        iteration_counter+=1
        print("Iteration: ", iteration_counter)
        print("Costo miglior soluzione: ", h.cost(instance, s_best))

    return s_best

no perturbation

In [15]:
def SISSRs_no_pert(instance, sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma, n_nearest, delta, epsilon, iter_imp, iter_max, p_min, p_max, max_unfeasible_swaps_perturb):
    s_0 = h.initial_solution_construction(instance, w1, w2, w3, w4, w5, gamma, n_nearest)
    s_curr = s_0
    s_best = s_0
    iterations_without_improvement = 0
    iteration_counter = 0
    while (iteration_counter < iter_max):
        s = h.ruin_and_recreate(instance, copy.deepcopy(s_curr), sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma)
        if h.cost(instance, s) < h.cost(instance, s_curr)*(1+delta):
            s_curr = h.local_search(instance, s, n_nearest)
            if h.cost(instance, s_curr) < h.cost(instance, s_best):
                s_best = s_curr
                iterations_without_improvement = 0
            else:
                iterations_without_improvement += 1
        delta = delta * epsilon
        iteration_counter+=1
        print("Iteration: ", iteration_counter)
        print("Costo miglior soluzione: ", h.cost(instance, s_best))

    return s_best

no sweep removal

In [17]:
def ruin_and_recreate_no_sweep(instance, solution, sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma):
   
    solution = h.random_drone_customer_removal(solution, sigma)

    solution = h.string_removal(instance, solution, c_average_removed, L_max)

    solution = h.recreate(instance, solution, w1, w2, w3, w4, w5, gamma)

    return solution


In [18]:
def SISSRs_no_sweep(instance, sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma, n_nearest, delta, epsilon, iter_imp, iter_max, p_min, p_max, max_unfeasible_swaps_perturb):
    s_0 = h.initial_solution_construction(instance, w1, w2, w3, w4, w5, gamma, n_nearest)
    s_curr = s_0
    s_best = s_0
    iterations_without_improvement = 0
    iteration_counter = 0
    while (iteration_counter < iter_max):
        s = ruin_and_recreate_no_sweep(instance, copy.deepcopy(s_curr), sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma)
        if h.cost(instance, s) < h.cost(instance, s_curr)*(1+delta):
            s_curr = h.local_search(instance, s, n_nearest)
            if h.cost(instance, s_curr) < h.cost(instance, s_best):
                s_best = s_curr
                iterations_without_improvement = 0
            else:
                iterations_without_improvement += 1
        if iterations_without_improvement >= iter_imp:
            s_curr = h.perturbate(instance, s_curr, p_min, p_max, max_unfeasible_swaps_perturb)
            iterations_without_improvement = 0
        delta = delta * epsilon
        iteration_counter+=1
        print("Iteration: ", iteration_counter)
        print("Costo miglior soluzione: ", h.cost(instance, s_best))

    return s_best

no random removal

In [20]:
def ruin_and_recreate_no_rand(instance, solution, sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma):
   
    solution = h.sweep_removal_operator(instance, solution, sigma)

    solution = h.string_removal(instance, solution, c_average_removed, L_max)

    solution = h.recreate(instance, solution, w1, w2, w3, w4, w5, gamma)

    return solution


In [21]:
def SISSRs_no_rand(instance, sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma, n_nearest, delta, epsilon, iter_imp, iter_max, p_min, p_max, max_unfeasible_swaps_perturb):
    s_0 = h.initial_solution_construction(instance, w1, w2, w3, w4, w5, gamma, n_nearest)
    s_curr = s_0
    s_best = s_0
    iterations_without_improvement = 0
    iteration_counter = 0
    while (iteration_counter < iter_max):
        s = ruin_and_recreate_no_rand(instance, copy.deepcopy(s_curr), sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma)
        if h.cost(instance, s) < h.cost(instance, s_curr)*(1+delta):
            s_curr = h.local_search(instance, s, n_nearest)
            if h.cost(instance, s_curr) < h.cost(instance, s_best):
                s_best = s_curr
                iterations_without_improvement = 0
            else:
                iterations_without_improvement += 1
        if iterations_without_improvement >= iter_imp:
            s_curr = h.perturbate(instance, s_curr, p_min, p_max, max_unfeasible_swaps_perturb)
            iterations_without_improvement = 0
        delta = delta * epsilon
        iteration_counter+=1
        print("Iteration: ", iteration_counter)
        print("Costo miglior soluzione: ", h.cost(instance, s_best))

    return s_best